In [9]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from autogluon.tabular import TabularDataset, TabularPredictor


power_test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
building_df = pd.read_csv('./building_info.csv')
final_copy =pd.read_csv('./power_final.csv')
from datetime import datetime, timedelta

final_copy = final_copy.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
power_test = power_test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
# building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,holidays,hour,day,month,year,sin_time,cos_time,THI,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
building_df = building_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'

})

# 건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
# 강수량, 일조, 일사 결측치 0.0으로 채우기
final_copy['rainfall'].fillna(0.0, inplace=True)
final_copy['sunshine'].fillna(0.0, inplace=True)
final_copy['solar_radiation'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
final_copy['windspeed'].fillna(round(final_copy['windspeed'].mean(),2), inplace=True)
final_copy['humidity'].fillna(round(final_copy['humidity'].mean(),2), inplace=True)
final_copy['solar_power_capacity'] = final_copy['solar_power_capacity'].replace('-', 0)
final_copy['ess_capacity'] = final_copy['ess_capacity'].replace('-', 0)
final_copy['pcs_capacity'] = final_copy['pcs_capacity'].replace('-', 0)


# test + building_info
power_test = power_test.merge(building_df, on='building_number', how='inner')
power_test.head()
power_test['solar_power_capacity'] = power_test['solar_power_capacity'].replace('-', 0)
power_test['ess_capacity'] = power_test['ess_capacity'].replace('-', 0)
power_test['pcs_capacity'] = power_test['pcs_capacity'].replace('-', 0)
# 결측치 처리해주고 train이랑 최대한 컬럼 맞춰서 돌리면 됨

In [10]:
final_copy

,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,holidays,...,year,sin_time,cos_time,THI,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0.0,...,2022,0.000000,1.000000,49.6576,Other Buildings,110634.00,39570.00,0,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,0.0,...,2022,0.258819,0.965926,47.7625,Other Buildings,110634.00,39570.00,0,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,0.0,...,2022,0.500000,0.866025,47.2225,Other Buildings,110634.00,39570.00,0,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,1.0,...,2022,0.707107,0.707107,44.7856,Other Buildings,110634.00,39570.00,0,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,1.0,...,2022,0.866025,0.500000,49.0061,Other Buildings,110634.00,39570.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,NaN,...,2022,-0.965926,0.258819,63.6624,Hotel and Resort,57497.84,40035.23,0,0,0
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,NaN,...,2022,-0.866025,0.500000,62.4024,Hotel and Resort,57497.84,40035.23,0,0,0
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,NaN,...,2022,-0.707107,0.707107,64.1976,Hotel and Resort,57497.84,40035.23,0,0,0
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,NaN,...,2022,-0.500000,0.866025,65.0744,Hotel and Resort,57497.84,40035.23,0,0,0


In [11]:

def classify_weekdays(start_date, end_date, holidays=[]):
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    
    classification = []
    current_date = start
    
    while current_date <= end:
        if current_date.weekday() < 5:  # 0: 월요일, 1: 화요일, ..., 4: 금요일
            classification.append(0)  # 평일
        else:
            classification.append(1)  # 주말
        
        current_date += timedelta(days=1)
    
    for holiday in holidays:
        holiday_date = datetime.strptime(holiday, date_format)
        index = (holiday_date - start).days
        if 0 <= index < len(classification):
            classification[index] = 2  # 해당 날짜를 주말로 설정
    
    return classification


# # '2022-08-15'를 휴일로 추가
holidays_to_add = ['2022-08-15']

data = {'date': pd.date_range(start='2022-06-01', end='2022-08-24')}
train = pd.DataFrame(data)
t_data = {'date': pd.date_range(start='2022-08-25', end='2022-08-31')}
test = pd.DataFrame(t_data)

# 주말/평일 분류한 결과를 'holidays' 열에 추가
start_date = "2022-06-01"
end_date = "2022-08-24"
test_start_date = "2022-08-25"
test_end_date = "2022-08-31"
train['holidays'] = classify_weekdays(start_date, end_date,holidays_to_add)
final_copy['holidays'] = train['holidays']
test['holidays'] = classify_weekdays(test_start_date, test_end_date)
power_test['holidays'] = test['holidays']
#added
final_copy['date_time'] = pd.to_datetime(final_copy['date_time'], format='%Y%m%d %H')
power_test['date_time'] = pd.to_datetime(power_test['date_time'], format='%Y%m%d %H')

# date time feature 생성
final_copy['hour'] = final_copy['date_time'].dt.hour
final_copy['day'] = final_copy['date_time'].dt.day
final_copy['month'] = final_copy['date_time'].dt.month
final_copy['year'] = final_copy['date_time'].dt.year

power_test['hour'] = power_test['date_time'].dt.hour
power_test['day'] = power_test['date_time'].dt.day
power_test['month'] = power_test['date_time'].dt.month
power_test['year'] = power_test['date_time'].dt.year

final_copy['sin_time'] = np.sin(2*np.pi*final_copy.hour/24)
final_copy['cos_time'] = np.cos(2*np.pi*final_copy.hour/24)

power_test['sin_time'] = np.sin(2*np.pi*power_test.hour/24)
power_test['cos_time'] = np.cos(2*np.pi*power_test.hour/24)
final_copy['THI'] = 9/5*final_copy['temperature'] - 0.55*(1-final_copy['humidity']/100)*(9/5*final_copy['humidity']-26)+32
power_test['THI'] = 9/5*power_test['temperature'] - 0.55*(1-power_test['humidity']/100)*(9/5*power_test['humidity']-26)+32
##############################################################


final_copy = final_copy.drop(columns = ['sunshine', 'solar_radiation', 'rainfall', 'windspeed'])
power_test = power_test.drop(columns = ['rainfall', 'windspeed'])
train_data = TabularDataset(final_copy)
train_data.head()

label = 'power_consumption'
####################################################

predictor = TabularPredictor(label=label,problem_type='regression').fit(
                            train_data, presets = ['best_quality'],  auto_stack=True)

test_data = TabularDataset(power_test)

y_pred = predictor.predict(test_data)
x = pd.DataFrame(y_pred)

sample_submission['answer'] = x['power_consumption']
print(sample_submission)
sample_submission.to_csv('./baseline_submission_g.csv', index=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20230814_201734\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230814_201734\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Disk Space Avail:   29.42 GB / 511.33 GB (5.8%)
Train Data Rows:    204000
Train Data Columns: 18
Label Column: power_consumption
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5086.91 MB
	Train Data (Original)  Memory Usage: 60.41 MB (1.

         num_date_time       answer
0        1_20220825 00  1936.686768
1        1_20220825 01  1938.094971
2        1_20220825 02  1940.844116
3        1_20220825 03  1818.583008
4        1_20220825 04  1834.193848
...                ...          ...
16795  100_20220831 19  1531.083252
16796  100_20220831 20  1512.346680
16797  100_20220831 21   759.521912
16798  100_20220831 22   652.857178
16799  100_20220831 23   674.676880

[16800 rows x 2 columns]
